In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats


fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"

In [ ]:
year = 2023
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=year,
    season_type_all_star="Regular Season",
)
df2 = stats.get_data_frames()[0]
df2.insert(0,"GAME_ID",df2.pop("GAME_ID"))

In [ ]:
stats = leaguedashteamstats.LeagueDashTeamStats(measure_type_detailed_defense="Advanced")
df1 = stats.get_data_frames()[0]
df1 = df1.sort_values("W_PCT").reset_index(drop=True)

In [ ]:
teams_1 = df1["TEAM_ID"].iloc[:6].to_list()
teams_2 = df1["TEAM_ID"].iloc[6:24].to_list()
teams_3 = df1["TEAM_ID"].iloc[24:].to_list()

In [ ]:
df1.iloc[6:24]

In [ ]:
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df_teams = df_teams.rename(columns={"nameTeam":"TEAM_NAME"})

In [ ]:
df3 = pd.merge(df1,df_teams,on="TEAM_NAME")
df3.index +=1

In [ ]:
df4 = df2.groupby("GAME_ID")
df4_1 = df4.nth(0)
df4_2 = df4.nth(1)
df4_1.columns = ["GAME_ID"] + [s + "1" for s in df4_1.columns if s != "GAME_ID"]
df4_2.columns = ["GAME_ID"] + [s + "2" for s in df4_2.columns if s != "GAME_ID"]
df4_3 = pd.merge(df4_1, df4_2, on="GAME_ID")
df4_4 = df4.nth(1)
df4_5 = df4.nth(0)
df4_4.columns = ["GAME_ID"] + [s + "1" for s in df4_4.columns if s != "GAME_ID"]
df4_5.columns = ["GAME_ID"] + [s + "2" for s in df4_5.columns if s != "GAME_ID"]
df4_6 = pd.merge(df4_4, df4_5, on="GAME_ID")
df5 = pd.concat([df4_3,df4_6])

In [ ]:
df5["W"] = df5["WL1"] == "W"
df5["L"] = df5["WL1"] != "W"
df6_1 = df5[df5["TEAM_ID2"].isin(teams_1)]
df6_2 = df5[df5["TEAM_ID2"].isin(teams_2)]
df6_3 = df5[df5["TEAM_ID2"].isin(teams_3)]

In [ ]:
df7_1 = df6_1.groupby("TEAM_NAME1")[["W","L"]].agg({"W":["sum"],"L":["sum"]}).reset_index()
df7_1.columns = ["TEAM_NAME","W","L"]
df7_1["W_PCT"] = df7_1["W"]/(df7_1["W"]+df7_1["L"])
df7_1["W_PCT"] = df7_1["W_PCT"].round(3)
df7_1 = df7_1.sort_values(["W_PCT","W"],ascending=False).reset_index(drop=True)
# df7_1 = pd.merge(df7_1,df_teams[["TEAM_NAME","urlThumbnailTeam"]],on=["TEAM_NAME"])
df7_1 = df7_1.reset_index()
df7_1["index"] +=1
df7_2 = df6_2.groupby("TEAM_NAME1")[["W","L"]].agg({"W":["sum"],"L":["sum"]}).reset_index()
df7_2.columns = ["TEAM_NAME","W","L"]
df7_2["W_PCT"] = df7_2["W"]/(df7_2["W"]+df7_2["L"])
df7_2["W_PCT"] = df7_2["W_PCT"].round(3)
df7_2 = df7_2.sort_values(["W_PCT","W"],ascending=False).reset_index(drop=True)
# df7_2 = pd.merge(df7_2,df_teams[["TEAM_NAME","urlThumbnailTeam"]],on=["TEAM_NAME"])
df7_2 = df7_2.reset_index()
df7_2["index"] +=1
df7_3 = df6_3.groupby("TEAM_NAME1")[["W","L"]].agg({"W":["sum"],"L":["sum"]}).reset_index()
df7_3.columns = ["TEAM_NAME","W","L"]
df7_3["W_PCT"] = df7_3["W"]/(df7_3["W"]+df7_3["L"])
df7_3["W_PCT"] = df7_3["W_PCT"].round(3)
df7_3 = df7_3.sort_values(["W_PCT","W"],ascending=False).reset_index(drop=True)
df7_3 = pd.merge(df7_3,df_teams[["TEAM_NAME","urlThumbnailTeam"]],on=["TEAM_NAME"])
df7_3 = df7_3.reset_index()
df7_3["index"] +=1

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i df7_3
df <- df7_3
library(tidyverse)
library(gt)
# library(gtExtras)
df %>% 
  gt()%>%
  text_transform(
      locations = cells_body(
        c(urlThumbnailTeam)
      ),
      fn = function(x) {
        web_image(
          url = x,
          height = 25
        )
      }
  ) %>%
  tab_header(
    title = md("**Team Records against Top 6 Teams**"),
    subtitle = md("Top 6 Teams wrt Win %"),
    ) %>%
    cols_label(
      index = "#", TEAM_NAME = "Team", W_PCT = "Win %", urlThumbnailTeam=""
    ) %>%
    # fmt_image(
    #   columns=urlThumbnailTeam
    # ) %>%
    
    # gt_img_rows(
    #   columns = urlThumbnailTeam, img_source = "web"
    # ) %>%
    cols_move(
      columns=urlThumbnailTeam,
      after=index
    ) %>%
    fmt_percent(
      columns=W_PCT,
      decimals=1
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 16,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        # data_row.padding = px(2),
        table.margin.left = 1,
        table.margin.right = 1,
    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_ratings/record_3.png",size=5) 